In [91]:
from pathlib import Path
import os
import numpy as np
import quaternion
import pandas as pd
from parse import parse
from Salient360Toolbox import helper
from Salient360Toolbox.generation import scanpath as scanp_generate
from Salient360Toolbox.generation import saliency as sal_generate
from scipy.spatial.transform import Rotation

In [92]:
# Tracking can be HE (Head+Eye) or H (Head alone)
tracking = "HE"

# Targeted eye
eye = "B"

# Resampling rate
resample = 200

# Filter settings
filterSettings = {"name": "savgol", "params": {"win": 9, "poly": 2}}

# Gaze parsing settings
parsingSettings = {"name": "I-VT", "params": {"threshold": 120}}

# Dimensions of output images (Height, Width)
dim = [8192, 16384]

# Path to CSV file containing raw gaze data
path_raw_file = r"experiment_data\raw_data.csv"

# Path to image stimulus
path_stim = r"experiment_data\image.jpg"

In [93]:
# load head and eye movement 
df = pd.read_csv(path_raw_file, sep = ";")
df.replace("INVALID", np.nan, inplace = True)
df.dropna(inplace = True, ignore_index = True)

initial_time = df.iloc[0,1]

# return absolute time in miliseconds
def absolute_time(x):
    return (x - initial_time) * 1e-6

df["ts"] = df["CaptureTime"].map(absolute_time)

In [94]:
# extract head rotations from the logfile and make four new comlumns in df containing the quaternion values
head_rotations = df["HMDRotation"]
parsed_headrotations = [parse("({x}, {y}, {z}, {w})", head_rotation).named for head_rotation in head_rotations]
headrotations = pd.DataFrame(parsed_headrotations)

df["xhead"] = headrotations["x"]
df["yhead"] = headrotations["y"]
df["zhead"] = headrotations["z"]
df["whead"] = headrotations["w"]

In [95]:
# extract right eye directions from the logfile and make three new comlumns in df containing the direction values
reye_directions = df["RightEyeForward"]
parsed_reyedirections = [parse("({x}, {y}, {z})", reye_direction).named for reye_direction in reye_directions]
df_reyedirections = pd.DataFrame(parsed_reyedirections)
df_reyedirections = pd.DataFrame(df_reyedirections.astype(float).to_numpy(),columns=["x","y","z"])

df["rightgazex"] = df_reyedirections["x"]
df["rightgazey"] = df_reyedirections["y"]
df["rightgazez"] = df_reyedirections["z"]

In [96]:
# extract left eye directions from the logfile and make three new comlumns in df containing the direction values
leye_directions = df["LeftEyeForward"]
parsed_leyedirections = [parse("({x}, {y}, {z})", leye_direction).named for leye_direction in leye_directions]
df_leyedirections = pd.DataFrame(parsed_leyedirections)
df_leyedirections = pd.DataFrame(df_leyedirections.astype(float).to_numpy(),columns=["x","y","z"])


df["leftgazex"] = df_leyedirections["x"]
df["leftgazey"] = df_leyedirections["y"]
df["leftgazez"] = df_leyedirections["z"]

In [97]:
# extract mean eye directions from the logfile and make three new comlumns in df containing the direction values
beye_directions = df["CombinedGazeForward"]
parsed_beyedirections = [parse("({x}, {y}, {z})", beye_direction).named for beye_direction in beye_directions]
df_beyedirections = pd.DataFrame(parsed_beyedirections)
df_beyedirections = pd.DataFrame(df_beyedirections.astype(float).to_numpy(),columns=["x","y","z"])


df["meangazedirx"] = df_beyedirections["x"]
df["meangazediry"] = df_beyedirections["y"]
df["meangazedirz"] = df_beyedirections["z"]

df.to_csv('experiment_data/gazelog.csv',columns=[ "ts", "xhead", "yhead", "zhead", "whead", "rightgazex", "rightgazey", "rightgazez", "leftgazex", "leftgazey", "leftgazez", "meangazedirx", "meangazediry", "meangazedirz"])

In [98]:
path = r"experiment_data\gazelog.csv"

# Get processed raw data and list of fix/sacc features 
gaze_data, fix_list = helper.loadRawData(path,
                                         
    # If gaze tracking, which eye to extract 
    eye = eye,
                                         
    # Gaze or Head tracking 
    tracking = tracking,
                                         
    # Resampling at a different sample rate? 
    resample = resample,
                                         
    # Filtering algo and parameters if any is selected 
    filter = filterSettings,
                                         
    # Fixation identifier algo and its parameters 
    parser = parsingSettings)

Loading the following columns from raw data files:
['ts', 'whead', 'xhead', 'yhead', 'zhead', 'meangazedirx', 'meangazediry', 'meangazedirz', 'meangazedirx', 'meangazediry', 'meangazedirz']
  Identifying saccades. It may take a minute or two.


In [99]:
savename = "saliencyexample"
PATH_OUT = r"experiment_data\processed_gaze\\"
outpath  = r"experiment_data\processed_gaze\\"
sal_map = helper.getSaliencyMap(fix_list[:, [2,3,4, 0,1]],
    dim,
    # Name of binary saliency file created for caching purposes
        name=savename,
        # If a binary file exists at this location we load the saliency data from it, unless force_generate is True. Saliency will be saved if caching is True
        path_save=PATH_OUT, # Sigma of the 2D Gaussian drawn at the location of fixations
        gauss_sigma=2, # Asks to return saliency data rather than a path to a saliency data file if it exists
        force_return_data=True, # Generate data instead of reading from pre-existing file
        force_generate=False, # Will save saliency to binary file to fast load at a later time
        caching=True)

Computing saliency data


In [100]:
fix_map = helper.getFixationMap(fix_list[:, :2], dim)

sal_image = sal_generate.toImage(sal_map, cmap="coolwarm")

# (fig 2.a) Save fixation map as a gray scale image
fix_map_img = sal_generate.toImage(fix_map, cmap="binary", reverse=True)
sal_generate.saveImage(fix_map_img, outpath+"_fixmap")

# (fig 2.b) Save saliency map as greyscale image
sal_generate.saveImage(sal_image, outpath+"_salmap")

# (fig 2.c) Save saliency map blended with stimulus 
sal_generate.saveImage(sal_map, outpath+"_bsalmap", blend=path_stim)

# (fig 2.d) Save stimulus with fixation points drawn over it
scanp_generate.toImage(fix_list[:, :2], dim, outpath+"_bscanpath", blend=path_stim)

# Save scanpath data (fixation and saccade features) to file
scanp_generate.toFile(fix_list, outpath+"_fixation.csv", 
                      # Save all features 
                      saveArr=np.arange(fix_list.shape[1]), mode="w")

In [101]:
np.count_nonzero(fix_map)

18